In [ ]:
from sim_util import preview_from_definition
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import colors
import gputools
import os
from skimage.io import imsave

%matplotlib inline
plt.rcParams['figure.figsize'] = [16,16]


# set and check which gpu is used
gputools.init_device(id_platform=0, id_device=1)
gputools.get_device()

In [ ]:
# simulate one plane of images
z_plane = 512

# for all parameter files in this dir:
param_basedir = '/Volumes/davidh-ssd/mv-sim/preview2'
params = [f for f in os.listdir(param_basedir) if f.endswith('.json')]

previews = {}
for p in params:
    print('simulating {}.'.format(p))
    preview = preview_from_definition(os.path.join(param_basedir, p), z_plane)
    previews[p] = preview
    print('{} done.'.format(p))
print('all done.')

In [ ]:
# plot the planes
# for different lambda, illum, x_pos

for p_file, preview in previews.items():

    for k in sorted(set(map(lambda x: x[::2], preview.keys()))):
        lam, xpos = k
        print(p_file, k)
        k_right = (lam, True, xpos)
        k_left = (lam, False, xpos)
        v_r = preview[k_right]
        v_l = preview[k_left]

        plt.figure()
        sig_r, desc_r = v_r
        sig_l, desc_l = v_l
        
        imsave(os.path.join(param_basedir, p_file + 'sig_l.tif'), sig_l.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'sig_r.tif'), sig_r.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'desc_l.tif'), desc_l.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'desc_r.tif'), desc_r.squeeze().astype(np.float32))
        
        sig_rgb = np.stack((sig_r.squeeze(), sig_l.squeeze(), np.zeros_like(sig_r.squeeze())), 2) 
        # crude intensity scaling
        sig_rgb *= 14
        plt.imshow(sig_rgb)
        plt.show()

        